.

## Average Values

Create a table filled with average values for heuristics of each class.


### Labels

#### RFI

* 1 : Unclipped RFI/Noise
* 2 : Wide-band, long-duration in time clipped RFI (2016+)
* 3 : Wide-band, short-duration in time clipped RFI (2016+)
* 4 : Wide-band, short duration clipped RFI (2015)
* 5 : Sharp bandpass transition
* 6 : Wide-band, bursty clipped RFI (2015)
* 7 : Error in spectra captue or replacement
* 8 : Systematic int/float overflow

#### Astronomical

* 0 : Interesting, follow up
* 9 : Known Pulsar

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Preamble

Read in the dataframe and prepare the features for use in scikit-learn.

In [27]:
pd.__version__

u'0.22.0'

In [28]:
df = pd.read_pickle('featureDateframe.pkl')

In [29]:
df.describe()

,Beam,Buffer,MJDstart,bestDM,bestSNR,BinFactor,Events,DMmax,DMmin,DMmean,...,pixelsMean133,pixelsMean140,pixelsMean141,pixelsMean142,pixelsMean143,pixelsMean150,pixelsMean151,pixelsMean152,pixelsMean153,DMin
count,73282.000000,73282.000000,73282.000000,73282.000000,73282.000000,73282.000000,7.328200e+04,73282.000000,73282.000000,73282.000000,...,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,1.553600e+04,15536.0
mean,3.556617,211.437952,57392.275750,994.132829,12.890546,14.171406,5.290837e+03,1843.879770,355.848537,1106.626399,...,-7.649538e+34,-1.288323e+34,1.429098e+34,-8.257392e+34,-5.166311e+34,-1.171005e+34,1.968355e+34,-5.147866e+34,-5.536758e+34,0.0
std,2.433997,272.753575,186.092812,2270.499049,79.265038,20.233713,3.448143e+04,3329.930528,1195.291547,2135.608269,...,1.325614e+36,2.036459e+36,2.037379e+36,1.691754e+36,1.241378e+36,1.969270e+36,2.078970e+36,1.700915e+36,1.240187e+36,0.0
min,0.000000,1.000000,57197.378446,0.000000,6.001704,1.000000,1.000000e+00,3.000000,0.000000,3.000000,...,-9.207459e+37,-6.056628e+37,-4.619998e+37,-3.490131e+37,-2.546661e+37,-5.667321e+37,-7.581628e+37,-3.265455e+37,-2.027739e+37,0.0
25%,1.000000,25.000000,57275.173537,7.000000,10.510364,2.000000,4.000000e+00,10.000000,5.000000,7.500000,...,0.000000e+00,-4.959372e+01,-1.515740e+01,-1.521857e+00,0.000000e+00,-4.950695e+01,-1.517410e+01,-1.548867e+00,0.000000e+00,0.0
50%,4.000000,111.000000,57328.330289,13.000000,11.301913,4.000000,3.400000e+01,27.000000,7.000000,16.500000,...,1.086369e+01,-1.265199e+01,-3.927948e+00,6.237355e+00,1.093944e+01,-1.249311e+01,-3.930597e+00,6.197421e+00,1.063596e+01,0.0
75%,6.000000,295.000000,57372.219326,273.000000,12.641252,16.000000,4.890000e+02,2024.000000,16.000000,947.812319,...,4.499946e+01,0.000000e+00,1.774699e+00,1.754257e+01,4.509341e+01,0.000000e+00,1.815087e+00,1.765943e+01,4.493125e+01,0.0
max,6.000000,2001.000000,57893.840891,10039.000000,20954.304688,64.000000,2.135578e+06,57287.182376,10035.000000,10035.500000,...,2.705174e+37,5.516305e+37,3.653238e+37,7.613141e+37,8.167050e+37,5.213531e+37,4.942519e+37,8.182356e+37,8.167050e+37,0.0


In [30]:
df.shape

(73282, 407)

#### Re-label Mis-labelled Type 8 Events

In [31]:
reLabel = [('Beam4_fb_D20150821T195709.fil', 202),
           ('Beam0_fb_D20150906T194609.fil', 275),
           ('Beam4_fb_D20150909T235709.fil', 467),
           ('Beam6_fb_D20150925T221909.fil', 3),
           ('Beam1_fb_D20151215T221703.fil', 377),
           ('Beam1_fb_D20151215T221703.fil', 595),
           ('Beam1_fb_D20151216T231103.fil', 50),
           ('Beam1_fb_D20151217T050412.fil', 196),
           ('Beam1_fb_D20151217T050412.fil', 197),
           ('Beam1_fb_D20151217T050412.fil', 202),
           ('Beam0_fb_D20160824T104004.fil', 1392),
           ('Beam0_fb_D20160825T003010.fil', 3),
           ('Beam0_fb_D20160824T104004.fil', 1392),
           ('Beam0_fb_D20160825T003010.fil', 3),
           ('Beam0_fb_D20170310T020409.fil', 6),
           ('Beam0_fb_D20170310T020409.fil', 8),
           ('Beam0_fb_D20170310T020409.fil', 9),
           ('Beam0_fb_D20170310T020409.fil', 11),
           ('Beam0_fb_D20170310T020409.fil', 14),
           ('Beam0_fb_D20170310T020409.fil', 15),
           ('Beam0_fb_D20170310T020409.fil', 16),
           ('Beam0_fb_D20170310T020409.fil', 17),
           ('Beam0_fb_D20170310T020409.fil', 19),
           ('Beam0_fb_D20170310T020409.fil', 21),
           ('Beam0_fb_D20170310T020409.fil', 25),
           ('Beam1_fb_D20170310T020409.fil', 4),
           ('Beam4_fb_D20170310T020409.fil', 7),
           ('Beam5_fb_D20170310T020409.fil', 1),
           ('Beam5_fb_D20170310T020409.fil', 2),
           ('Beam6_fb_D20170311T184903.fil', 2),
           ('Beam1_fb_D20170312T182504.fil', 2),
           ('Beam2_fb_D20170325T011509.fil', 1),
           ('Beam1_fb_D20170310T020409.fil', 3),
           ('Beam0_fb_D20170311T184903.fil', 2),
           ('Beam1_fb_D20170311T184903.fil', 1),
           ('Beam5_fb_D20170312T182504.fil', 1),
           ('Beam1_fb_D20170310T020409.fil', 2),
           ('Beam6_fb_D20170311T184903.fil', 3),
           ('Beam4_fb_D20170312T182504.fil', 10),
           ('Beam0_fb_D20170310T020409.fil', 2),
           ('Beam4_fb_D20170310T020409.fil', 4),
           ('Beam0_fb_D20170310T020409.fil', 1),
           ('Beam0_fb_D20170310T020409.fil', 3),
           ('Beam0_fb_D20170310T020409.fil', 4),
           ('Beam0_fb_D20170310T020409.fil', 5),
           ('Beam0_fb_D20170310T020409.fil', 7),
           ('Beam0_fb_D20170310T020409.fil', 10),
           ('Beam0_fb_D20170310T020409.fil', 12),
           ('Beam0_fb_D20170310T020409.fil', 13),
           ('Beam0_fb_D20170310T020409.fil', 18),
           ('Beam0_fb_D20170310T020409.fil', 20),
           ('Beam0_fb_D20170310T020409.fil', 22),
           ('Beam0_fb_D20170310T020409.fil', 23),
           ('Beam0_fb_D20170310T020409.fil', 24),
           ('Beam1_fb_D20170310T020409.fil', 1),
           ('Beam4_fb_D20170310T020409.fil', 3),
           ('Beam4_fb_D20170310T020409.fil', 5),
           ('Beam4_fb_D20170310T020409.fil', 6),
           ('Beam0_fb_D20170311T184903.fil', 1)]

In [32]:
# Relabel mis-labelled events to overflow (label 8)
for fil,buf in reLabel:
    #print fil, buf
    df.loc[(df['filterbank']==fil) & (df['Buffer']==buf), 'Label'] = 8

#### Compute MJD Features

We don't care about the absolute MJDs, but the relative offset within the dataset.

In [33]:
df['relMJDmax'] = df['MJDmax'] - df['MJDmin']
df['relMJDmean'] = df['MJDmean'] - df['MJDmin']
df['relMJDmedian'] = df['MJDmedian'] - df['MJDmin']

In [35]:
# df.reset_index(drop=True, inplace=True)

# #Create a dataframe with only float values, convert to a numpy array:
# floatdf = df.drop(['Beam', 'Buffer', 'TSID', 'Label', 'datfile', 'predictLabel', 'filterbank', 'MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)
# fullarr = floatdf.values.astype('float32')

# #Find row indices where values are extremely large:
# overflowargs = np.argwhere(fullarr > 1e+7)
# overflowidx = np.hsplit(overflowargs,2)[0]
# overflowidx = np.unique(overflowidx)
# overflowidx.flatten()
# print overflowidx.shape

# #Put overflows in label 8:
# for i in overflowidx:
#     df.ix[i, 'Label'] = 8
#     #print i

# #Drop overflow values:
# fullar = df.drop(overflowidx)

# fullarr = floatdf.values.astype('float32')
# print np.amax(fullarr)

/home/inigo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  


(245649, 1)
(1604,)
nan


**Filter for only labelled data (i.e. Label > -1) and Drop Type 8 Events**

In [9]:
labelledDf = df[df['Label'] > -1] 
unlabelledDf = df[df['Label'] < -0.5]
labelledDf = labelledDf[labelledDf['Label'] != 8] #delete column labelled by 8

**Relabel 0 to 9**

We are interested in both 9 and 0 labels and both appear similar.

In [10]:
labelledDf.loc[labelledDf['Label'] == 0, 'Label'] = 9 #finds indices where label is 0 or 9

In [11]:
for col in labelledDf:
    if labelledDf[col].dtype == object:
        print col, labelledDf[col].min(), labelledDf[col].max() 
    else: 
        print col, labelledDf[col].min(), labelledDf[col].max(), labelledDf[col].std()

datfile Beam0_dm_D20150624T020452.dat Beam6_dm_D20170324T215304.dat
Beam 0 6 2.64289416941
TSID D20150624T020452 D20170325T092403
Buffer 1 1997 234.099730177
MJDstart 57197.3848466 57837.3807827 158.637469447
bestDM 51.0 10039.0 2925.42377249
bestSNR 10.0002756119 181.342330933 7.78511888853
BinFactor 1 64 23.3707904092
Events 1 1800508 63404.711047
DMmax 54.0 10039.0 3741.79111564
DMmin 0.0 10035.0 1854.50379848
DMmean 25.1612903226 10035.5 2705.60863642
DMmedian 9.0 10035.5 2819.01443151
DMstd 0.707106781187 4672.49363237 881.751739387
SNRmean 7.07141349051 19.9987541906 1.31552412109
SNRmedian 6.77030944824 15.0827293396 0.96828782134
SNRstd 0.000775502012695 14.4079302489 1.31336401293
MJDmax 57197.3848944 57837.3808056 158.637465108
MJDmin 57197.3848934 57837.3808055 158.637468173
MJDstd 0.0 4.11672955923e-05 8.34844478405e-06
MJDmean 57197.384894 57837.3808055 158.637466696
MJDmedian 57197.384894 57837.3808055 158.637466697
Label 1 9 2.14351963379
predictLabel -1 -1 0.0
filterban

pixelsMax13 -22.8797588348 450.725006104 46.3924568073
pixelsMax20 -53.5337677002 283.089782715 40.8251187157
pixelsMax21 -19.0611896515 293.414276123 35.7069742617
pixelsMax22 -36.2150497437 1053.2154541 41.0400596338
pixelsMax23 -23.3056411743 411.588195801 46.4270973972
pixelsMax30 -54.1617355347 281.867675781 40.935575157
pixelsMax31 -18.9161815643 349.350402832 35.6163553819
pixelsMax32 -35.9291915894 1055.77722168 40.8041529768
pixelsMax33 -22.8020496368 402.516693115 46.4068010771
pixelsMax40 -55.2150764465 284.620483398 40.9806287441
pixelsMax41 -17.5338668823 335.453216553 35.9503859367
pixelsMax42 -36.1824302673 1047.36437988 41.0783714657
pixelsMax43 -22.7590503693 370.587432861 46.7201739977
pixelsMax50 -52.4001159668 284.637634277 41.1593885731
pixelsMax51 -19.9815731049 306.741516113 36.4807785542
pixelsMax52 -36.2249908447 1045.46972656 41.780893022
pixelsMax53 -22.935256958 336.706573486 47.2734853384
pixelsMax60 -54.4544715881 283.901855469 41.2599227023
pixelsMax61 -1

**Apply Direct Data Cleaning Filters**

In [12]:
# # Check that data during March 2017 has been removed
# for row in labelledDf.iterrows():
#     idx, data = row
#     if (data['globtsStatsMean'] < -3e3) or (data['globtsStatsMin'] < -3e3): 
#         print idx

# for row in unlabelledDf.iterrows():
#     idx, data = row
#     if (data['globtsStatsMean'] < -3e3) or (data['globtsStatsMin'] < -3e3): 
#         print idx
        
# # Check that all events which overflow have been dropped
# for row in labelledDf.iterrows():
#     idx, data = row
#     if data['ofCount'] > 0.: 
#         print idx

# for row in unlabelledDf.iterrows():
#     idx, data = row
#     if data['ofCount'] > 0.: 
#         print idx       

In [13]:
#(n timeseries, m features)
print labelledDf.shape
print unlabelledDf.shape

(13419, 410)
(58212, 410)


**Select out the labels**

In [14]:
labels = labelledDf['Label'].values

**Drop the meta-information columns**

In [15]:
labelledDf.columns #list column labels

Index([u'datfile', u'Beam', u'TSID', u'Buffer', u'MJDstart', u'bestDM',
       u'bestSNR', u'BinFactor', u'Events', u'DMmax',
       ...
       u'pixelsMean142', u'pixelsMean143', u'pixelsMean150', u'pixelsMean151',
       u'pixelsMean152', u'pixelsMean153', u'DMin', u'relMJDmax',
       u'relMJDmean', u'relMJDmedian'],
      dtype='object', length=410)

In [16]:
featureDf = labelledDf.drop(['Beam', 'Buffer', 'datfile', 'TSID', 'Label', 'predictLabel', 'filterbank'], axis=1)
unlblfeatureDf = unlabelledDf.drop(['Beam', 'Buffer', 'TSID', 'Label', 'datfile', 'predictLabel', 'filterbank'], axis=1)

#### Drop Features that are not useful

In [17]:
featureDf = featureDf.drop(['MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)
unlblfeatureDf = unlblfeatureDf.drop(['MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)

# featureDf.replace([np.inf, -np.inf], np.nan)
# unlblfeatureDf.replace([np.inf, -np.inf], np.nan)


# featureDf = featureDf.fillna(value=-1)
# unlblfeatureDf= unlblfeatureDf.fillna(value=-1)

# print featureDf.isnull().values.any()

In [18]:
featureDf.columns

Index([u'MJDstart', u'bestDM', u'bestSNR', u'BinFactor', u'Events', u'DMmax',
       u'DMmin', u'DMmean', u'DMmedian', u'DMstd',
       ...
       u'pixelsMean142', u'pixelsMean143', u'pixelsMean150', u'pixelsMean151',
       u'pixelsMean152', u'pixelsMean153', u'DMin', u'relMJDmax',
       u'relMJDmean', u'relMJDmedian'],
      dtype='object', length=399)

### Pre-Processing

Prepare the features for model building: outliers (inf, NaN) need to be replaces, the features need to be scaled to zero mean, and unity variance.

In [19]:
import sklearn.preprocessing

**Convert feature dataframe to 2-D array of floats**

In [20]:
featuresArr = featureDf.values.astype('float32')
unlblfeaturesArr = unlblfeatureDf.values.astype('float32')
preProcLabels = labels.copy()

In [25]:
print np.argwhere(unlblfeaturesArr > 1e+7).shape

/home/inigo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


(59, 2)


**Replace NaN and inf values**

In [21]:
featuresArr[featuresArr == -np.inf] = np.nan
featuresArr[featuresArr == np.inf] = np.nan #set infinite values to nan (could u use .isinf more easily here?)
print featuresArr[featuresArr == np.nan]

unlblfeaturesArr[unlblfeaturesArr == -1. * np.inf] = np.nan
unlblfeaturesArr[unlblfeaturesArr == np.inf] = np.nan
print unlblfeaturesArr[unlblfeaturesArr == np.nan]

#replace missing values with the mean along axis 0, np.nan values replaced with 'NaN' string
imp = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(featuresArr)
unlblimp = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(unlblfeaturesArr)#calibrate Imputer

preProcFeats = imp.transform(featuresArr)
unlblpreProcFeats = unlblimp.transform(unlblfeaturesArr)


print preProcFeats.shape, preProcLabels.shape
print np.amax(preProcFeats), np.amin(preProcFeats)
print np.amax(unlblpreProcFeats), np.amin(unlblpreProcFeats)

[]
[]
[]
(13419, 399) (13419,)
1.80051e+06 -15904.6
2.4457e+38 -1.78561e+38


**Scale Features**

Robust scaling uses a median filter, this is better for non-Gaussian distributions

In [22]:
rScale = sklearn.preprocessing.RobustScaler().fit(preProcFeats)
#print rScale.scale_
#print rScale.center_

preProcFeats = rScale.transform(preProcFeats)

print preProcFeats.shape

rScale = sklearn.preprocessing.RobustScaler().fit(unlblpreProcFeats)
#print rScale.scale_
#print rScale.center_

unlblpreProcFeats = rScale.transform(unlblpreProcFeats)

print unlblpreProcFeats.shape

(13419, 399)
(58212, 399)


In [23]:
print np.amax(preProcFeats), np.amin(preProcFeats)
print np.amax(unlblpreProcFeats), np.amin(unlblpreProcFeats)

5856.99 -222661.0
2.44046e+38 -1.78178e+38


#### Write important dataframes/arrays to files

In [24]:
np.save('./processed.files/labelled/features', preProcFeats)
np.save('./processed.files/labelled/labels', preProcLabels)
featureDf.to_pickle('./processed.files/labelled/featureDf')
labelledDf.to_pickle('./processed.files/labelled/labelledDf')

np.save('./processed.files/raw/features', unlblpreProcFeats)
unlblfeatureDf.to_pickle('./processed.files/raw/featureDf')
unlabelledDf.to_pickle('./processed.files/raw/unlabelledDf')